In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import ast
import numpy as np

In [2]:
runtimes_comparison = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Final Model Results\Comparison Results\Runtime Comparison.csv")

In [3]:
runtimes_comparison['num_row'] = runtimes_comparison['filename'].apply(lambda x: len(pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\CTGAN Synthetic Datasets\Source' + '\\' + x)))
runtimes_comparison['num_features'] = runtimes_comparison['filename'].apply(lambda x: len(pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\CTGAN Synthetic Datasets\Source' + '\\' + x).columns))
runtimes_comparison['num_ele'] = runtimes_comparison['num_row'] * runtimes_comparison['num_features']

In [152]:
runtimes_comparison.loc[runtimes_comparison['filename'] == 'occupancy.csv', 'knn'] =  579.030680

In [153]:
runtimes_comparison[runtimes_comparison['filename'] == 'occupancy.csv']

,filename,knn,smbo,rs,mfe,num_row,num_features,num_ele
69,occupancy.csv,579.03068,1190.864843,74.742249,0.091512,1944,6,11664


In [6]:
runtimes_comparison.loc[runtimes_comparison['smbo'].idxmax()]

filename        Airline Passenger Satisfaction.csv
knn                                      767.87235
smbo                                  16304.364435
rs                                      707.156621
mfe                                       2.819941
num_row                                     107904
num_features                                    23
num_ele                                    2481792
Name: 22, dtype: object

In [155]:
runtimes_comparison.to_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Final Model Results\Comparison Results\Runtime Comparison.csv", index=False)

In [7]:
np.mean(runtimes_comparison['knn'])

np.float64(374.98767869116386)

In [8]:
print(f"knn Mean Runtime: {np.mean(runtimes_comparison['knn'])}")
print(f"MFE Mean Runtime: {np.mean(runtimes_comparison['mfe'])}")
print(f"SMBO Mean Runtime: {np.mean(runtimes_comparison['smbo'])}")
print(f"Random Search Mean Runtime: {np.mean(runtimes_comparison['rs'])}")

knn Mean Runtime: 374.98767869116386
MFE Mean Runtime: 0.4964829404303368
SMBO Mean Runtime: 1908.3834924469602
Random Search Mean Runtime: 160.60432935014686


In [7]:
smbo_results = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\SMBO Results\All SMBO Results.csv")
rs_results = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Random Search Results\All Random Search Results.csv")
knn_results = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Final Model Results\kNN PCA kMedoids Test Results\Results\kNN All\All Results.csv")
mfe_results = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Final Model Results\Common MFE Test Results\Results\MFE All\All Results.csv")

In [143]:
knn_results

,filename,best_hp
0,air_quality.csv,"(3, (16, 16, 16))"
1,Algerian_forest_fires_dataset_UPDATE.csv,"(2, (16, 8, 0))"
2,Banknote.csv,"(3, (16, 16, 16))"
3,breast-cancer-wisconsin.csv,"(2, (1, 16, 0))"
4,cancer.csv,"(2, (16, 16, 0))"
...,...,...
93,wall_robot_navigation.csv,"(3, (16, 16, 16))"
94,Wilt.csv,"(3, (1, 16, 1))"
95,wine.csv,"(3, (8, 16, 16))"
96,winequality-white.csv,"(3, (1, 16, 16))"


In [9]:
len(pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\CTGAN Synthetic Datasets\Source\accident.csv'))

192

In [156]:
smbo_lm = LinearRegression()
smbo_data = runtimes_comparison[['filename', 'num_row', 'num_features', 'num_ele', 'smbo']]
smbo_data.columns = ['filename', 'num_row', 'num_features', 'num_ele', 'runtime']
X_smbo = smbo_data.iloc[:, 1:-1]
y_smbo = smbo_data['runtime']

smbo_lm.fit(X_smbo, y_smbo)

LinearRegression()

In [163]:
smbo_data['pred'] = smbo_lm.predict(X_smbo) *1.2
smbo_data['result'] = smbo_data['pred'] >= smbo_data['runtime']
smbo_data['result'].value_counts()

C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\1961234314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smbo_data['pred'] = smbo_lm.predict(X_smbo) *1.2
C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\1961234314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smbo_data['result'] = smbo_data['pred'] >= smbo_data['runtime']


result
True    94
Name: count, dtype: int64

In [162]:
smbo_data[smbo_data['result'] == False]

,filename,num_row,num_features,num_ele,runtime,pred,result
38,dataset_185_yeast.csv,1342,9,12078,1193.567752,1193.347995,False
92,winequality-white.csv,3304,12,39648,1508.175804,1507.270151,False
93,yeast.csv,1342,9,12078,1218.521120,1193.347995,False


In [164]:
rs_lm = LinearRegression()
rs_data = runtimes_comparison[['filename', 'num_row', 'num_features', 'num_ele', 'rs']]
rs_data.columns = ['filename', 'num_row', 'num_features', 'num_ele', 'runtime']
X_rs = rs_data.iloc[:, 1:-1]
y_rs = rs_data['runtime']

rs_lm.fit(X_rs, y_rs)

LinearRegression()

In [169]:
rs_data['pred'] = rs_lm.predict(X_rs) * 2.2
rs_data['result'] = rs_data['pred'] >= rs_data['runtime']
rs_data['result'].value_counts()

C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\1229996237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rs_data['pred'] = rs_lm.predict(X_rs) * 2.2
C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\1229996237.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rs_data['result'] = rs_data['pred'] >= rs_data['runtime']


result
True    94
Name: count, dtype: int64

In [167]:
rs_data[rs_data['result']==False]

,filename,num_row,num_features,num_ele,runtime,pred,result
25,banana_quality.csv,7988,8,63904,336.478270,321.013847,False
26,Bank Customer Churn Prediction.csv,4074,10,40740,309.636136,299.864477,False
27,BHP Attack.csv,830,21,17430,301.541282,274.604082,False
65,merged_vehicle_silhouettes.csv,832,19,15808,298.900369,275.519641,False


In [103]:
rs_data

,filename,num_row,num_features,num_ele,runtime,pred,result
0,air_quality.csv,4000,10,40000,103.448602,329.340964,True
1,Algerian_forest_fires_dataset_UPDATE.csv,209,14,2926,114.395276,300.426157,True
2,Banknote.csv,1220,5,6100,114.560597,312.011914,True
3,breast-cancer-wisconsin.csv,478,10,4780,124.617002,305.069588,True
4,cancer.csv,424,31,13144,136.371505,291.532770,True
...,...,...,...,...,...,...,...
89,wall_robot_navigation.csv,4850,25,121250,101.770784,347.572410,True
90,Wilt.csv,522,6,3132,73.093610,307.751784,True
91,wine.csv,118,14,1652,70.994225,299.679645,True
92,winequality-white.csv,3304,12,39648,91.936643,325.180182,True


In [170]:
knn_lm = LinearRegression()
knn_data = runtimes_comparison[['filename', 'num_row', 'num_features', 'num_ele', 'knn']]
knn_data.columns = ['filename', 'num_row', 'num_features', 'num_ele', 'runtime']
X_knn = knn_data.iloc[:, 1:-1]
y_knn = knn_data['runtime']

knn_lm.fit(X_knn, y_knn)

LinearRegression()

In [177]:
knn_data['pred'] = knn_lm.predict(X_knn) * 2.5
knn_data['result'] = knn_data['pred'] >= knn_data['runtime']
knn_data['result'].value_counts()

C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\3209407192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knn_data['pred'] = knn_lm.predict(X_knn) * 2.5
C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\3209407192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knn_data['result'] = knn_data['pred'] >= knn_data['runtime']


result
True    94
Name: count, dtype: int64

In [178]:
knn_data[knn_data['result']==False]

,filename,num_row,num_features,num_ele,runtime,pred,result


In [126]:
knn_data

,filename,num_row,num_features,num_ele,runtime,pred,result
0,air_quality.csv,4000,10,40000,290.118610,785.457869,True
1,Algerian_forest_fires_dataset_UPDATE.csv,209,14,2926,250.359362,754.105953,True
2,Banknote.csv,1220,5,6100,260.980848,805.467223,True
3,breast-cancer-wisconsin.csv,478,10,4780,277.802348,777.495393,True
4,cancer.csv,424,31,13144,292.102770,659.720703,True
...,...,...,...,...,...,...,...
89,wall_robot_navigation.csv,4850,25,121250,268.412987,732.003026,True
90,Wilt.csv,522,6,3132,267.343251,799.835658,True
91,wine.csv,118,14,1652,351.471996,753.743081,True
92,winequality-white.csv,3304,12,39648,260.304414,775.165990,True


In [179]:
mfe_lm = LinearRegression()
mfe_data = runtimes_comparison[['filename', 'num_row', 'num_features', 'num_ele', 'mfe']]
mfe_data.columns = ['filename', 'num_row', 'num_features', 'num_ele', 'runtime']
X_mfe = mfe_data.iloc[:, 1:-1]
y_mfe = mfe_data['runtime']

mfe_lm.fit(X_mfe, y_mfe)

LinearRegression()

In [182]:
mfe_data['pred'] = mfe_lm.predict(X_mfe) * 1.6
mfe_data['result'] = mfe_data['pred'] >= mfe_data['runtime']
mfe_data['result'].value_counts()

C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\3700629567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfe_data['pred'] = mfe_lm.predict(X_mfe) * 1.6
C:\Users\Matthew Wijaya\AppData\Local\Temp\ipykernel_19852\3700629567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfe_data['result'] = mfe_data['pred'] >= mfe_data['runtime']


result
True     93
False     1
Name: count, dtype: int64

In [133]:
mfe_data[mfe_data['result']==False]

,filename,num_row,num_features,num_ele,runtime,pred,result
30,Click Prediction.csv,30114,12,361368,8.488929,1.409945,False
42,EEG Eye State.csv,13446,15,201690,1.127996,1.076498,False


In [85]:
mfe_data

,filename,num_row,num_features,runtime,pred,result
0,air_quality.csv,4000,10,0.385656,10.356683,True
1,Algerian_forest_fires_dataset_UPDATE.csv,209,14,0.438493,10.400674,True
2,Banknote.csv,1220,5,0.099506,10.145639,True
3,breast-cancer-wisconsin.csv,478,10,0.246165,10.283582,True
4,cancer.csv,424,31,0.531479,10.926507,True
...,...,...,...,...,...,...
89,wall_robot_navigation.csv,4850,25,0.762738,10.834358,True
90,Wilt.csv,522,6,0.097603,10.161820,True
91,wine.csv,118,14,0.416308,10.398786,True
92,winequality-white.csv,3304,12,0.532640,10.403575,True


In [183]:
def predict_runtime(algo, num_row, num_features):
    if algo == 'smbo':
        return 1.2 * smbo_lm.predict(np.array([num_row, num_features]))
    if algo == 'rs':
        return rs_lm.predict(np.array([num_row, num_features])) * 2.2
    if algo == 'knn':
        return knn_lm.predict(np.array([num_row, num_features])) * 2.5
    if algo == 'mfe':
        return mfe_lm.predict(np.array([num_row, num_features])) * 1.6

In [5]:
mfe_features = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Final Model Results\Common MFE Features\Features\All Common MFE.csv", index_col=0)

In [7]:
mfe_features.columns

Index(['filename', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin',
       'nr_inst', 'nr_num', 'cor.mean', 'cor.sd', 'cov.mean', 'cov.sd',
       'eigenvalues.mean', 'eigenvalues.sd', 'g_mean.mean', 'g_mean.sd',
       'gravity', 'h_mean.mean', 'h_mean.sd', 'iq_range.mean', 'iq_range.sd',
       'kurtosis.mean', 'kurtosis.sd', 'mad.mean', 'mad.sd', 'nr_cor_attr',
       'nr_norm', 'nr_outliers', 'skewness.mean', 'skewness.sd',
       'sparsity.mean', 'sparsity.sd', 't_mean.mean', 't_mean.sd',
       'attr_conc.mean', 'attr_conc.sd', 'attr_ent.mean', 'attr_ent.sd',
       'class_conc.mean', 'class_conc.sd', 'class_ent', 'eq_num_attr',
       'joint_ent.mean', 'joint_ent.sd', 'mut_inf.mean', 'mut_inf.sd',
       'ns_ratio'],
      dtype='object')

In [40]:
def make_mfe_df(feature_names, family, definitions):
    df = pd.DataFrame(columns=['Meta-feature Name', 'Family', 'Definition'])
    for i in range(len(feature_names)):
        new_row = [feature_names[i], family, definitions[i]]
        # print(new_row)
        df.loc[i] = new_row
    
    return df

In [41]:
general_def = ['Ratio between the number of attributes', 'Ratio between the number of instances and attributes', 'Total number of features', 'Number of binary features', 
               'Number of rows in dataset', 'Number of numeric features']
general_df = make_mfe_df(['attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin','nr_inst', 'nr_num'], 'General', general_def)

In [42]:
general_df

,Meta-feature Name,Family,Definition
0,attr_to_inst,General,Ratio between the number of attributes
1,inst_to_attr,General,Ratio between the number of instances and attr...
2,nr_attr,General,Total number of features
3,nr_bin,General,Number of binary features
4,nr_inst,General,Number of rows in dataset
5,nr_num,General,Number of numeric features


In [43]:
statistical_cols = ['cor', 'cov', 'eigenvalues', 'g_mean', 'gravity', 'h_mean', 'iq_range', 'kurtosis', 'mad', 'nr_cor_attr',
       'nr_norm', 'nr_outliers', 'skewness', 'sparsity', 't_mean']
statistical_def = ['Absolute value of the correlation of distinct dataset column pairs', 'Absolute value of the covariance of distinct dataset attribute pairs', 
                   'Eigenvalues of covariance matrix from dataset', 'Geometric mean of each attribute', 'Distance between minority and majority classes center of mass',
                   'Harmonic mean of each attribute', 'Interquartile range (IQR) of each attribute', 'Kurtosis of each attribute', 'Median Absolute Deviation (MAD) adjusted by a factor',
                   'Number of distinct highly correlated pair of attributes', 'Number of attributes normally distributed based in a given method', 
                   'Number of attributes with at least one outlier value', 'Skewness for each attribute', 'Sparsity metric for each attribute', 'Trimmed mean of each attribute']
statistical_df = make_mfe_df(statistical_cols, 'Statistical', statistical_def)

In [44]:
statistical_df

,Meta-feature Name,Family,Definition
0,cor,Statistical,Absolute value of the correlation of distinct ...
1,cov,Statistical,Absolute value of the covariance of distinct d...
2,eigenvalues,Statistical,Eigenvalues of covariance matrix from dataset
3,g_mean,Statistical,Geometric mean of each attribute
4,gravity,Statistical,Distance between minority and majority classes...
5,h_mean,Statistical,Harmonic mean of each attribute
6,iq_range,Statistical,Interquartile range (IQR) of each attribute
7,kurtosis,Statistical,Kurtosis of each attribute
8,mad,Statistical,Median Absolute Deviation (MAD) adjusted by a ...
9,nr_cor_attr,Statistical,Number of distinct highly correlated pair of a...


In [45]:
mfe_features.columns

Index(['filename', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin',
       'nr_inst', 'nr_num', 'cor.mean', 'cor.sd', 'cov.mean', 'cov.sd',
       'eigenvalues.mean', 'eigenvalues.sd', 'g_mean.mean', 'g_mean.sd',
       'gravity', 'h_mean.mean', 'h_mean.sd', 'iq_range.mean', 'iq_range.sd',
       'kurtosis.mean', 'kurtosis.sd', 'mad.mean', 'mad.sd', 'nr_cor_attr',
       'nr_norm', 'nr_outliers', 'skewness.mean', 'skewness.sd',
       'sparsity.mean', 'sparsity.sd', 't_mean.mean', 't_mean.sd',
       'attr_conc.mean', 'attr_conc.sd', 'attr_ent.mean', 'attr_ent.sd',
       'class_conc.mean', 'class_conc.sd', 'class_ent', 'eq_num_attr',
       'joint_ent.mean', 'joint_ent.sd', 'mut_inf.mean', 'mut_inf.sd',
       'ns_ratio'],
      dtype='object')

In [46]:
info_cols = ['attr_conc', 'attr_ent', 'class_conc', 'class_ent', 'eq_num_attr', 'joint_ent', 'mut_inf', 'ns_ratio']
info_def = ['Concentration coefficient of each pair of distinct attributes', 'Shannon’s entropy for each predictive attribute', 'Concentration coefficient between each attribute and class',
            'Target attribute Shannon’s entropy', 'Number of attributes equivalent for a predictive task', 'Joint entropy between each attribute and class',
            'Mutual information between each attribute and target', 'Noisiness of attributes']
info_df = make_mfe_df(info_cols, 'Information Theory', info_def)

In [47]:
info_df

,Meta-feature Name,Family,Definition
0,attr_conc,Information Theory,Concentration coefficient of each pair of dist...
1,attr_ent,Information Theory,Shannon’s entropy for each predictive attribute
2,class_conc,Information Theory,Concentration coefficient between each attribu...
3,class_ent,Information Theory,Target attribute Shannon’s entropy
4,eq_num_attr,Information Theory,Number of attributes equivalent for a predicti...
5,joint_ent,Information Theory,Joint entropy between each attribute and class
6,mut_inf,Information Theory,Mutual information between each attribute and ...
7,ns_ratio,Information Theory,Noisiness of attributes


In [50]:
mfe_info_df = pd.concat([general_df, statistical_df, info_df], ignore_index=True)

In [53]:
print(mfe_info_df.to_latex(index=False))

\begin{tabular}{lll}
\toprule
Meta-feature Name & Family & Definition \\
\midrule
attr_to_inst & General & Ratio between the number of attributes \\
inst_to_attr & General & Ratio between the number of instances and attributes \\
nr_attr & General & Total number of features \\
nr_bin & General & Number of binary features \\
nr_inst & General & Number of rows in dataset \\
nr_num & General & Number of numeric features \\
cor & Statistical & Absolute value of the correlation of distinct dataset column pairs \\
cov & Statistical & Absolute value of the covariance of distinct dataset attribute pairs \\
eigenvalues & Statistical & Eigenvalues of covariance matrix from dataset \\
g_mean & Statistical & Geometric mean of each attribute \\
gravity & Statistical & Distance between minority and majority classes center of mass \\
h_mean & Statistical & Harmonic mean of each attribute \\
iq_range & Statistical & Interquartile range (IQR) of each attribute \\
kurtosis & Statistical & Kurtosis of ea